In [1]:
import pandas as pd

data = pd.read_excel('task2_data_processing.xlsx', sheet_name=None)
print(data)


{'Conflicting Roles':    S.no    Base Role                    Conflicting Role Conflict Role ID
0     1  A/P Invoice  Adding Customer/Vendor Master Data          CRID001
1     2  A/P Invoice                      Adding Lead BP          CRID002
2     3  A/P Invoice             Remove Business Partner          CRID003, 'Access Listing - Validation':        User     Department  ...      Adding Lead BP Remove Business Partner
0   User AV  Manufacturing  ...  Full Authorization      Full Authorization
1    User A        Support  ...  Full Authorization      Full Authorization
2   User AT        Finance  ...    No Authorization        No Authorization
3   User AQ  Manufacturing  ...  Full Authorization      Full Authorization
4    User M  Manufacturing  ...  Full Authorization      Full Authorization
5   User AR  Manufacturing  ...    No Authorization        No Authorization
6    User K             HR  ...    No Authorization        No Authorization
7   User AM        Support  ...    No Auth

In [2]:
data.keys()

dict_keys(['Conflicting Roles', 'Access Listing - Validation', 'Invoice Validation'])

In [3]:
sheet1 = data['Conflicting Roles']
sheet2 = data['Access Listing - Validation']
sheet3 = data['Invoice Validation']

In [4]:
filter_dataframe = pd.merge(sheet3, sheet2, on=['User'], how='inner')
filter_dataframe

,Document Number,Creation Date,Document Type,Document Status,User,Invoice Value,Department,A/P Invoice,Adding Customer/Vendor Master Data,Adding Lead BP,Remove Business Partner
0,1819100458,2019-01-08,Item,Closed,User A,18849,Support,Full Authorization,Full Authorization,Full Authorization,Full Authorization
1,1819100563,2019-01-30,Item,Closed,User A,92040,Support,Full Authorization,Full Authorization,Full Authorization,Full Authorization
2,1819100589,2019-01-30,Item,Closed,User A,187640,Support,Full Authorization,Full Authorization,Full Authorization,Full Authorization
3,1819100681,2019-02-04,Service,Closed,User A,623,Support,Full Authorization,Full Authorization,Full Authorization,Full Authorization
4,1819100733,2019-02-19,Service,Closed,User A,2818,Support,Full Authorization,Full Authorization,Full Authorization,Full Authorization
...,...,...,...,...,...,...,...,...,...,...,...
994,1919000537,2019-03-20,Service,Open,User I,8625,Manufacturing,Full Authorization,Full Authorization,No Authorization,No Authorization
995,1927000288,2019-03-04,Item,Closed,User I,18480,Manufacturing,Full Authorization,Full Authorization,No Authorization,No Authorization
996,1927000328,2019-03-20,Item,Closed,User I,44755,Manufacturing,Full Authorization,Full Authorization,No Authorization,No Authorization
997,1927000277,2019-03-02,Service,Closed,User AJ,6705,Manufacturing,Read-Only,No Authorization,No Authorization,No Authorization


In [5]:
filter_dataframe['Authorization'] = 'No'

In [6]:
filter_dataframe.loc[(filter_dataframe['A/P Invoice'] == 'Full Authorization') & (filter_dataframe['Adding Customer/Vendor Master Data'] == 'No Authorization') & 
                                                              (filter_dataframe['Adding Lead BP'] == 'No Authorization') &
                                                              (filter_dataframe['Remove Business Partner'] == 'No Authorization'), 'Authorization'] = 'Yes'

In [7]:
filter_dataframe

,Document Number,Creation Date,Document Type,Document Status,User,Invoice Value,Department,A/P Invoice,Adding Customer/Vendor Master Data,Adding Lead BP,Remove Business Partner,Authorization
0,1819100458,2019-01-08,Item,Closed,User A,18849,Support,Full Authorization,Full Authorization,Full Authorization,Full Authorization,No
1,1819100563,2019-01-30,Item,Closed,User A,92040,Support,Full Authorization,Full Authorization,Full Authorization,Full Authorization,No
2,1819100589,2019-01-30,Item,Closed,User A,187640,Support,Full Authorization,Full Authorization,Full Authorization,Full Authorization,No
3,1819100681,2019-02-04,Service,Closed,User A,623,Support,Full Authorization,Full Authorization,Full Authorization,Full Authorization,No
4,1819100733,2019-02-19,Service,Closed,User A,2818,Support,Full Authorization,Full Authorization,Full Authorization,Full Authorization,No
...,...,...,...,...,...,...,...,...,...,...,...,...
994,1919000537,2019-03-20,Service,Open,User I,8625,Manufacturing,Full Authorization,Full Authorization,No Authorization,No Authorization,No
995,1927000288,2019-03-04,Item,Closed,User I,18480,Manufacturing,Full Authorization,Full Authorization,No Authorization,No Authorization,No
996,1927000328,2019-03-20,Item,Closed,User I,44755,Manufacturing,Full Authorization,Full Authorization,No Authorization,No Authorization,No
997,1927000277,2019-03-02,Service,Closed,User AJ,6705,Manufacturing,Read-Only,No Authorization,No Authorization,No Authorization,No


In [8]:
print(filter_dataframe['Authorization'].value_counts())

Yes    688
No     311
Name: Authorization, dtype: int64


In [9]:
def conflict(row):
  res = []
  if row['Authorization'] == 'No':
    if row['A/P Invoice'] != "Full Authorization":
      res.append(row['A/P Invoice'])
    if row['Adding Customer/Vendor Master Data']=="Full Authorization":
      res.append("CRID001")
    if row['Adding Lead BP']=="Full Authorization":
      res.append("CRID002")
    if row['Remove Business Partner']=="Full Authorization":
      res.append("CRID003")
  else:
    return "No Conflicting Rules"
  return ",".join(res)

In [10]:
filter_dataframe['Conflicting Rules'] = filter_dataframe.apply(conflict,axis = 1)

In [11]:
filter_dataframe['Conflicting Rules'].value_counts()

No Conflicting Rules                688
CRID001,CRID002,CRID003              84
Read-Only                            82
CRID002,CRID003                      54
CRID003                              42
No Authorization,CRID002             38
No Authorization,CRID001,CRID003      7
CRID001                               4
Name: Conflicting Rules, dtype: int64

In [12]:
final = filter_dataframe[['Document Number','Invoice Value','Department','Authorization','Conflicting Rules']]
final

,Document Number,Invoice Value,Department,Authorization,Conflicting Rules
0,1819100458,18849,Support,No,"CRID001,CRID002,CRID003"
1,1819100563,92040,Support,No,"CRID001,CRID002,CRID003"
2,1819100589,187640,Support,No,"CRID001,CRID002,CRID003"
3,1819100681,623,Support,No,"CRID001,CRID002,CRID003"
4,1819100733,2818,Support,No,"CRID001,CRID002,CRID003"
...,...,...,...,...,...
994,1919000537,8625,Manufacturing,No,CRID001
995,1927000288,18480,Manufacturing,No,CRID001
996,1927000328,44755,Manufacturing,No,CRID001
997,1927000277,6705,Manufacturing,No,Read-Only


In [13]:
final.to_csv('output.csv',index=False)